# 用户和事件关联关系处理


整个数据集中事件数目（events.csv）太多，所以下面的处理我们找出只在训练集和测试集中出现的事件和用户集合，并对他们重新编制索引

In [1]:
#保存数据
import pickle

import itertools

#处理事件字符串
import datetime

import numpy as np
import scipy.io as sio
import scipy.sparse as ss

#相似度/距离
import scipy.spatial.distance as ssd

from collections import defaultdict
from sklearn.preprocessing import normalize

In [2]:
 """
我们只关心train和test中出现的user和event，因此重点处理这部分关联数据

train.csv 有6列：
user：用户ID
event：事件ID
invited：是否被邀请（0/1）
timestamp：ISO-8601 UTC格式时间字符串，表示用户看到该事件的时间
interested, and not_interested

Test.csv 除了没有interested, and not_interested，其余列与train相同
 """
    
# 统计训练集中有多少独立的用户的events
uniqueUsers = set()
uniqueEvents = set()

#统计每个用户参加的事件   / 每个事件参加的用户
eventsForUser = defaultdict(set)
usersForEvent = defaultdict(set)
    
for filename in ["train.csv", "test.csv"]:
    f = open(filename, 'rb')
    
    #忽略第一行（列名字）
    f.readline().strip().decode().split(",")
    
    for line in f:    #对每条记录
        cols = line.strip().decode().split(",")
        uniqueUsers.add(cols[0])   #第一列为用户ID
        uniqueEvents.add(cols[1])   #第二列为事件ID
        eventsForUser[cols[0]].add(cols[1])    #该用户参加了这个事件
        usersForEvent[cols[1]].add(cols[0])    #该事件被用户参加
    f.close()


n_uniqueUsers = len(uniqueUsers)
n_uniqueEvents = len(uniqueEvents)

print("number of uniqueUsers :%d" % n_uniqueUsers)
print("number of uniqueEvents :%d" % n_uniqueEvents)

##统计每个用户参加的事件，后续用于将用户朋友参加的活动影响到用户
pickle.dump(eventsForUser, open("PE_eventsForUser.pkl", 'wb'))
##统计事件参加的用户
pickle.dump(usersForEvent, open("PE_usersForEvent.pkl", 'wb'))

#用户关系矩阵表，可用于后续LFM/SVD++处理的输入
#这是一个稀疏矩阵，记录用户对事件感兴趣
userEventScores = ss.dok_matrix((n_uniqueUsers, n_uniqueEvents))
userIndex = dict()
eventIndex = dict()

#重新编码用户索引字典
for i, u in enumerate(uniqueUsers):
    userIndex[u] = i
    
#重新编码用户索引字典    
for i, e in enumerate(uniqueEvents):
    eventIndex[e] = i
    
ftrain = open("train.csv", 'rb')
ftrain.readline()
for line in ftrain:
    cols = line.strip().decode().split(",")
    i = userIndex[cols[0]]  #用户
    j = eventIndex[cols[1]] #事件
    userEventScores[i, j] = int(cols[4]) - int(cols[5])   #interested - not_interested
ftrain.close()
  
#保存用户-事件关系矩阵R，以备后用
sio.mmwrite("PE_userEventScores", userEventScores)

#保存用户索引表
pickle.dump(userIndex, open("PE_userIndex.pkl", 'wb'))
#保存事件索引表
pickle.dump(eventIndex, open("PE_eventIndex.pkl", 'wb'))

    
# 为了防止不必要的计算，我们找出来所有关联的用户 或者 关联的event
# 所谓的关联用户，指的是至少在同一个event上有行为的用户pair
# 关联的event指的是至少同一个user有行为的event pair
uniqueUserPairs = set()
uniqueEventPairs = set()
for event in uniqueEvents:
    users = usersForEvent[event]
    if len(users) > 2:
        uniqueUserPairs.update(itertools.combinations(users, 2))
        
for user in uniqueUsers:
    events = eventsForUser[user]
    if len(events) > 2:
        uniqueEventPairs.update(itertools.combinations(events, 2))
 
#保存用户-事件关系对索引表
pickle.dump(uniqueUserPairs, open("PE_uniqueUserPairs.pkl", 'wb'))
pickle.dump(uniqueEventPairs, open("PE_uniqueEventPairs.pkl", 'wb'))

number of uniqueUsers :3391
number of uniqueEvents :13418


In [3]:
userIndex

{'2348719537': 0,
 '1214006960': 1,
 '2255113076': 2,
 '544449933': 3,
 '3153655575': 4,
 '499079668': 5,
 '292322045': 6,
 '2242768450': 7,
 '23150045': 8,
 '2796038131': 9,
 '2301083010': 10,
 '1652448351': 11,
 '128581247': 12,
 '2187693519': 13,
 '1543396188': 14,
 '745331470': 15,
 '2022846287': 16,
 '4148656943': 17,
 '3308369328': 18,
 '635462786': 19,
 '1349737346': 20,
 '1873358601': 21,
 '4238321438': 22,
 '3833491880': 23,
 '149914134': 24,
 '754938428': 25,
 '2658254174': 26,
 '2493566825': 27,
 '3148555709': 28,
 '3170052426': 29,
 '3475765358': 30,
 '192951763': 31,
 '516110926': 32,
 '351313256': 33,
 '3663559651': 34,
 '2560745948': 35,
 '4189881681': 36,
 '3828329584': 37,
 '825875823': 38,
 '814715283': 39,
 '2624862533': 40,
 '1614557413': 41,
 '3749087176': 42,
 '2282422131': 43,
 '4165509784': 44,
 '283716061': 45,
 '4106927042': 46,
 '307271704': 47,
 '3488552280': 48,
 '2243065045': 49,
 '2360121322': 50,
 '2621463744': 51,
 '458848274': 52,
 '2593432040': 53,
 '

In [5]:
#训练集和测试集中出现的用户数目和事件数目远小于users.csv出现的用户数和events.csv出现的事件数